In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import mixed_precision
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [2]:
# from google.colab import drive
# import gdown

# # Mount Google Drive
# drive.mount('/content/drive')

# # Replace with your file ID from Google Drive
# file_id = "13t4sY57Pnd8ENG4ENU3YYFUTGXzJX4rf"

# # Specify the destination path within your Google Drive
# destination = '/content/drive/MyDrive/dataset.csv'  # Change this path as needed

# # Download the file from Google Drive
# gdown.download(f"https://drive.google.com/uc?id={file_id}", destination, quiet=False)


In [3]:
import os

# List the extracted files
os.listdir("/content/drive/MyDrive")


['Recipe Dataset',
 'Colab Notebooks',
 'dataset.zip',
 'dataset.csv',
 'datasets',
 'food_model.h5',
 'food_model_float32.h5',
 'dataset.csv5c6dtdqt.part',
 'yolov5-master',
 'fridge.webp',
 'yolov5']

recipe dataset loading

In [4]:
import pandas as pd

# Load the CSV file from Google Drive
df = pd.read_csv('/content/drive/MyDrive/dataset.csv')

# Display the first few rows of the dataset
df.head()


,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [5]:
# import gdown
# import zipfile

# # Download URL for Food-101
# url = "https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz"
# output = "food-101.tar.gz"

# # Download and extract
# gdown.download(url, output, quiet= False)


In [6]:
# import tarfile

# # Path to the downloaded file
# tar_file_path = 'food-101.tar.gz'

# # Extract the tar.gz file
# with tarfile.open(tar_file_path, 'r:gz') as tar:
#     tar.extractall(path='/content/drive/MyDrive/datasets/food101')  # Change the path if necessary


foof 101 dataset loading

In [7]:
import os

# Path where you extracted the dataset
extracted_path = '/content/drive/MyDrive/datasets/food101/food-101'  # Change this to your desired path

# Verify the extraction of Food-101
print("Food-101 dataset extracted to:", extracted_path)
print("Contents of the extracted folder:", os.listdir(extracted_path))


Food-101 dataset extracted to: /content/drive/MyDrive/datasets/food101/food-101
Contents of the extracted folder: ['images', 'meta', 'license_agreement.txt', 'README.txt']


data preprocessing, including rescaling and splitting the dataset into training and validation subsets.

In [8]:
from tensorflow.keras.callbacks import EarlyStopping
df['ingredients'] = df['ingredients'].apply(lambda x: x.lower().strip())
def extract_ingredients(ingredient_str):
    return ingredient_str.split(',')
df['ingredient_list'] = df['ingredients'].apply(extract_ingredients)
food_images_path = os.path.join(extracted_path, 'images')
food_classes = os.listdir(food_images_path)

# Image Data Generator
image_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming `X` contains images and `y` contains labels.
# Split into 90% training+validation and 10% testing
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Further split the remaining 90% into 80% training and 10% validation
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1111, random_state=42)  # 0.1111 ensures ~10% of total

print(f"Training data: {len(X_train)}")
print(f"Validation data: {len(X_val)}")
print(f"Testing data: {len(X_test)}")


In [ ]:
# Training and Validation Generators
image_gen = ImageDataGenerator(rescale=1./255)

train_gen = image_gen.flow(X_train, y_train, batch_size=32)
val_gen = image_gen.flow(X_val, y_val, batch_size=32)

# Testing (scaled manually since ImageDataGenerator is not applied)
X_test = X_test / 255.0


In [9]:
train_gen = image_gen.flow_from_directory(
    food_images_path,
    target_size=(128, 128),
    batch_size=16,  # Reduced batch size
    class_mode='categorical',
    subset='training'
)

val_gen = image_gen.flow_from_directory(
    food_images_path,
    target_size=(128, 128),
    batch_size=16,  # Reduced batch size
    class_mode='categorical',
    subset='validation'
)

Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


Build the CNN Model

In [10]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Load a pre-trained MobileNetV2 model and freeze its layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False  # Freeze the pre-trained model's layers

# Build the final model
model = models.Sequential([
    base_model,  # Pre-trained MobileNetV2 as feature extractor
    layers.GlobalAveragePooling2D(),  # Pooling layer to reduce spatial dimensions
    layers.Dense(256, activation='relu'),  # Dense layer for higher-level features
    layers.Dense(len(food_classes), activation='softmax')  # Output layer with number of food classes
])

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model using the ImageDataGenerator's train and validation generators
history = model.fit(
    train_gen,  # Training data generator
    epochs=10,  # Number of epochs
    validation_data=val_gen,  # Validation data generator
    callbacks=[early_stopping],  # Early stopping to avoid overfitting
    steps_per_epoch=50,  # Limit steps per epoch for faster training
    validation_steps=10  # Limit validation steps to speed up the process
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


50/50 ━━━━━━━━━━━━━━━━━━━━ 101s 2s/step - accuracy: 0.0138 - loss: 4.9402 - val_accuracy: 0.0625 - val_loss: 4.3666
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.0800 - loss: 4.2886 - val_accuracy: 0.1250 - val_loss: 4.0366
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.1484 - loss: 3.8696 - val_accuracy: 0.2125 - val_loss: 3.4730
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 86s 2s/step - accuracy: 0.1617 - loss: 3.6121 - val_accuracy: 0.2500 - val_loss: 3.3046
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.2387 - loss: 3.3956 - val_accuracy: 0.2937 - val_loss: 3.0292
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.2595 - loss: 3.1715 - val_accuracy: 0.2750 - val_loss: 2.9913
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 85s 2s/step - accuracy: 0.2861 - loss: 2.9452 - val_accuracy: 0.3063 - val_loss: 2.9827
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.3495 - loss: 2.8371 - val_accuracy: 0.2812 - val_loss: 2.8861
Ep

In [79]:
pip install torch torchvision opencv-python


In [81]:
!git clone https://github.com/ultralytics/yolov5 /content/drive/MyDrive/yolov5


Cloning into '/content/drive/MyDrive/yolov5'...
remote: Enumerating objects: 17075, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 17075 (delta 19), reused 7 (delta 7), pack-reused 17049 (from 2)
Receiving objects: 100% (17075/17075), 15.68 MiB | 11.27 MiB/s, done.
Resolving deltas: 100% (11719/11719), done.
Updating files: 100% (146/146), done.


In [82]:
!pip install -U torch torchvision matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 28.5 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.8.0
    Uninstalling matplotlib-3.8.0:
      Successfully uninstalled matplotlib-3.8.0


In [11]:
import sys
sys.path.append('/content/drive/MyDrive/yolov5')  # Add YOLOv5 directory to Python path

import torch

# Load the YOLOv5 model (adjust the model size if needed, 'yolov5s' is the small version)
# Providing the 'yolov5' directory as the repo and specifying the model
yolov5_model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/yolov5/yolov5s.pt')
# or
# If you have cloned the repo, then you can directly load from the local directory
# yolov5_model = torch.hub.load('/content/drive/MyDrive/yolov5', 'yolov5s', source='local')

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-12-22 Python-3.10.12 torch-2.5.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [12]:
def detect_items_with_yolo(image_path, yolov5_model):
    img = cv2.imread(image_path)  # Read image using OpenCV
    results = yolov5_model(img)  # Run YOLOv5 detection
    results.print()  # Print detection results

    # Get detected items and their labels (class names)
    detected_items = results.names  # Labels of detected objects
    return detected_items

In [13]:
def predict_fridge_contents(image_path, model, label_encoder, img_size=(128, 128)):
    # Read and resize the image
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Image at path {image_path} could not be loaded. Check the path and file.")
    img_resized = cv2.resize(img, img_size)
    img_resized = np.expand_dims(img_resized / 255.0, axis=0)  # Normalize and expand dimensions

    # Predict the contents
    predictions = model.predict(img_resized)
    predicted_labels = label_encoder.inverse_transform(np.argsort(predictions[0])[-5:][::-1])  # Top 5 predictions
    return predicted_labels


In [15]:
label_encoder = LabelEncoder()
# You'll need to fit it to your class labels at some point before using it
# Assuming 'food_classes' from your earlier code contains the class labels:
label_encoder.fit(food_classes)

fridge_image_path = r"/content/drive/MyDrive/fridge.webp"

# Detect items using YOLOv5
detected_items = detect_items_with_yolo(fridge_image_path, yolov5_model)
print(f"Detected food items with YOLOv5: {detected_items}")

# Predict food items using the trained model (e.g., food classification model)
detected_food_items = predict_fridge_contents(fridge_image_path, model, label_encoder)  # Now using the created label_encoder
print(f"Predicted food items: {detected_food_items}")

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
image 1/1: 280x462 2 bottles, 1 bowl
Speed: 4.2ms pre-process, 410.6ms inference, 2.2ms NMS per image at shape (1, 3, 416, 640)


Detected food items with YOLOv5: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboar

In [ ]:

from sklearn.preprocessing import LabelEncoder
import ast
import gc
import psutil
import torch

# Monitor memory usage
def monitor_memory():
    memory = psutil.virtual_memory()
    print(f"Available memory: {memory.available / 1024 / 1024} MB")

# Parse ingredient strings into lists
def parse_ingredients(ingredient_str):
    try:
        return ast.literal_eval(ingredient_str)  # Safely evaluate the string to a list
    except:
        return []

# Suggest recipes based on detected items
def suggest_recipes(detected_items, recipes_df):
    detected_items = [item.strip().lower() for item in detected_items]  # Clean detected items
    suggested_recipes = []

    for _, recipe in recipes_df.iterrows():
        ingredients = [ingredient.strip().lower() for ingredient in recipe['ingredient_list']]
        if all(item in ingredients for item in detected_items):
            suggested_recipes.append(recipe['title'])

    return suggested_recipes

# Process recipes in batches to handle large datasets
def process_in_batches(detected_items, df, batch_size=1000):
    suggested_recipes_all = []
    for start in range(0, len(df), batch_size):
        batch = df[start:start + batch_size]
        suggested_recipes_batch = suggest_recipes(detected_items, batch)
        suggested_recipes_all.extend(suggested_recipes_batch)
        del batch
        gc.collect()
    return suggested_recipes_all

# Function to detect items in fridge image using YOLOv5
def detect_items_with_yolo(image_path, yolov5_model):
    img = cv2.imread(image_path)  # Read image using OpenCV
    results = yolov5_model(img)  # Run YOLOv5 detection
    detected_items = results.pandas().xyxy[0]['name'].tolist()  # Extract names of detected objects
    return detected_items


# Load the recipe dataset
df = pd.read_csv(dataset_path)
df['ingredient_list'] = df['ingredients'].apply(parse_ingredients)

# Detect items in the fridge
detected_items = detect_items_with_yolo(fridge_image_path, yolov5_model)
print(f"Detected items: {detected_items}")

# Suggest recipes based on detected items
monitor_memory()
suggested_recipes = process_in_batches(detected_items, df)
monitor_memory()

# Handle cases where no recipes are found
if suggested_recipes:
    print(f"Suggested recipes: {suggested_recipes}")
else:
    print("No matching recipes found for the detected items.")
